In [1]:
import locale

#from matplotlib import pyplot as plt
import numpy as np

from db import psqlServer
sv = psqlServer()

control_chars = [chr(0x85)]
locale.setlocale(locale.LC_ALL, 'C')



'C'

In [2]:
def normalize_text(text):
    norm_text = text.lower()
    # Replace breaks with spaces
    norm_text = norm_text.replace('<br />', ' ')
    # Pad punctuation with spaces on both sides
    for char in ['.', '"', ',', '(', ')', '!', '?', ';', ':']:
        norm_text = norm_text.replace(char, ' ' + char + ' ')
    return norm_text

In [3]:
AI = "arx_AI"        # artificial intelligence
FA = "arx_math_FA"   # functional analysis
GR = "arx_GRQC"      # general relativity and quantum cosmology
LG = "arx"           # learning
NT = "arx_math_NT"   # number theory

In [4]:
# How many items in all tables?
UNION = " UNION ALL "
SEL_C = "SELECT COUNT(*) as FOO FROM "
sum_statement = "SELECT SUM(FOO) FROM ("
sum_statement += SEL_C + AI
sum_statement += UNION + SEL_C + FA 
sum_statement += UNION + SEL_C + GR 
sum_statement += UNION + SEL_C + LG 
sum_statement += UNION + SEL_C + NT
sum_statement += ") as B;"

total = sv.execute(sum_statement)[0]['sum']
print(total)

116060


In [5]:
# Per table
SEL_C = "SELECT COUNT(*) FROM "
total_AI = sv.execute(SEL_C + AI)[0]['count']
total_FA = sv.execute(SEL_C + FA)[0]['count']
total_GR = sv.execute(SEL_C + GR)[0]['count']
total_LG = sv.execute(SEL_C + LG)[0]['count']
total_NT = sv.execute(SEL_C + NT)[0]['count']

print("AI:{}".format(total_AI))
print("FA:{}".format(total_FA))
print("GR:{}".format(total_GR))
print("LG:{}".format(total_LG))
print("NT:{}".format(total_NT))

AI:13000
FA:16000
GR:50000
LG:15060
NT:22000


In [6]:
# Get some rows from the AI table
rows = sv.execute("SELECT * FROM arx_math_NT")
corpus = [x['abstract'] for x in rows]
labels = [float(x['has_journal_ref']) for x in rows]

#rows2 = sv.execute("SELECT abstract FROM arx_math_FA LIMIT 10000")
#corpus += [x['abstract'] for x in rows2]
#labels += [x['has_journal_ref'] for x in rows2]

#rows3 = sv.execute("SELECT abstract FROM arx_GRQC LIMIT 30000")
#corpus += [x['abstract'] for x in rows3]
#labels += [x['has_journal_ref'] for x in rows2]


In [7]:
corpus_n = []
jj = 0
for corp in corpus:
    corpus_n.append(u"_*{0} {1}\n".format(jj, normalize_text(corp)))
    jj += 1

In [8]:
len(corpus_n)

22000

In [9]:
import gensim
from collections import namedtuple


In [10]:
SentimentDocument = namedtuple('SentimentDocument', 'words tags split sentiment')
alldocs = []
for line_no, line in enumerate(corpus_n[0:5500]):
    tokens = gensim.utils.to_unicode(corpus_n[line_no]).split()
    words = tokens[1:]
    tags = [line_no]
    split = 'train'
    #split = ['train', 'test', 'extra', 'extra'][line_no//12500]
    sentiment = labels[line_no]
    #sentiment = [1.0, 0.0, 1.0, 0.0, None, None, None, None][line_no//6750]
    alldocs.append(SentimentDocument(words, tags, split, sentiment))

for line_no, line in enumerate(corpus_n[5500:11000]):
    tokens = gensim.utils.to_unicode(corpus_n[line_no]).split()
    words = tokens[1:]
    tags = [line_no]
    split = 'test'
    #split = ['train', 'test', 'extra', 'extra'][line_no//12500]
    sentiment = labels[line_no]
    #sentiment = [1.0, 0.0, 1.0, 0.0, None, None, None, None][line_no//6750]
    alldocs.append(SentimentDocument(words, tags, split, sentiment))
    
for line_no, line in enumerate(corpus_n[11000:]):
    tokens = gensim.utils.to_unicode(corpus_n[line_no]).split()
    words = tokens[1:]
    tags = [line_no]
    split = 'extra'
    #split = ['train', 'test', 'extra', 'extra'][line_no//12500]
    sentiment = None
    #sentiment = [1.0, 0.0, 1.0, 0.0, None, None, None, None][line_no//6750]
    alldocs.append(SentimentDocument(words, tags, split, sentiment))    

In [11]:
len(alldocs)

22000

In [12]:
train_docs = [doc for doc in alldocs if doc.split == 'train']
test_docs = [doc for doc in alldocs if doc.split == 'test']
doc_list = alldocs[:]  # For reshuffling per pass

print('%d docs: %d train-sentiment, %d test-sentiment' % (len(doc_list), len(train_docs), len(test_docs)))

22000 docs: 5500 train-sentiment, 5500 test-sentiment


In [13]:
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from collections import OrderedDict
import multiprocessing

In [14]:
cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

In [15]:
simple_models = [
    # PV-DM w/ concatenation - window=5 (both sides) approximates paper's 10-word total window size
    Doc2Vec(dm=1, dm_concat=1, size=100, window=5, negative=5, hs=0, min_count=2, workers=cores),
    # PV-DBOW 
    Doc2Vec(dm=0, size=100, negative=5, hs=0, min_count=2, workers=cores),
    # PV-DM w/ average
    Doc2Vec(dm=1, dm_mean=1, size=100, window=10, negative=5, hs=0, min_count=2, workers=cores),
]

In [16]:
simple_models[0].build_vocab(alldocs)

In [17]:
print(simple_models[0])
for model in simple_models[1:]:
    """Reuse shareable structures from other_model."""
    model.reset_from(simple_models[0])
    print(model)

Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8)
Doc2Vec(dbow,d100,n5,mc2,s0.001,t8)
Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8)


In [18]:
models_by_name = OrderedDict((str(model), model) for model in simple_models)
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([simple_models[1], simple_models[2]])
models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([simple_models[1], simple_models[0]])

In [19]:
models_by_name

OrderedDict([('Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8)',
              <gensim.models.doc2vec.Doc2Vec at 0x10d2410b8>),
             ('Doc2Vec(dbow,d100,n5,mc2,s0.001,t8)',
              <gensim.models.doc2vec.Doc2Vec at 0x10d241198>),
             ('Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8)',
              <gensim.models.doc2vec.Doc2Vec at 0x10d241240>),
             ('dbow+dmm',
              <gensim.test.test_doc2vec.ConcatenatedDoc2Vec at 0x1144ca240>),
             ('dbow+dmc',
              <gensim.test.test_doc2vec.ConcatenatedDoc2Vec at 0x1144ca208>)])

In [20]:

# Predictive Evaluation Methods

# Let's define some helper methods for evaluating the performance of our Doc2vec
# using paragraph vectors. We will classify document sentiments using a logistic 
# regression model based on our paragraph embeddings. We will compare the error 
# rates based on word embeddings from our various Doc2vec models.
import numpy as np
import statsmodels.api as sm
from random import sample

# For timing
from contextlib import contextmanager
from timeit import default_timer
import time 


/Users/alex/code/article-classifier/ac/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [21]:
@contextmanager
def elapsed_timer():
    start = default_timer()
    elapser = lambda: default_timer() - start
    yield lambda: elapser()
    end = default_timer()
    elapser = lambda: end-start
    
def logistic_predictor_from_data(train_targets, train_regressors):
    logit = sm.Logit(train_targets, train_regressors)
    predictor = logit.fit(disp=0)
    # print(predictor.summary())
    return predictor

def error_rate_for_model(test_model, train_set, test_set, infer=False, infer_steps=3, infer_alpha=0.1, infer_subsample=0.1):
    """Report error rate on test_doc sentiments, using supplied model and train_docs"""

    train_targets, train_regressors = zip(*[(doc.sentiment, test_model.docvecs[doc.tags[0]]) for doc in train_set])
    train_regressors = sm.add_constant(train_regressors)
    predictor = logistic_predictor_from_data(train_targets, train_regressors)

    test_data = test_set
    if infer:
        if infer_subsample < 1.0:
            test_data = sample(test_data, int(infer_subsample * len(test_data)))
        test_regressors = [test_model.infer_vector(doc.words, steps=infer_steps, alpha=infer_alpha) for doc in test_data]
    else:
        test_regressors = [test_model.docvecs[doc.tags[0]] for doc in test_docs]
    test_regressors = sm.add_constant(test_regressors)
    
    # Predict & evaluate
    test_predictions = predictor.predict(test_regressors)
    
    # Here is the wtf: test_data is just [None]*500!!!
    corrects = sum(np.rint(test_predictions) == [doc.sentiment for doc in test_data])
    errors = len(test_predictions) - corrects
    error_rate = float(errors) / len(test_predictions)
    return (error_rate, errors, len(test_predictions), predictor)

In [22]:

# Bulk Training

# We use an explicit multiple-pass, alpha-reduction approach as sketched in 
# this gensim doc2vec blog post with added shuffling of corpus on each pass.
# https://rare-technologies.com/doc2vec-tutorial/
from collections import defaultdict
best_error = defaultdict(lambda: 1.0)  # To selectively print only best errors achieved

In [23]:
from random import shuffle
import datetime

alpha, min_alpha, passes = (0.025, 0.001, 10)
alpha_delta = (alpha - min_alpha) / passes

print("START %s" % datetime.datetime.now())

for epoch in range(passes):
    shuffle(doc_list)  # Shuffling gets best results
    
    for name, train_model in models_by_name.items():
        # Train
        duration = 'na'
        train_model.alpha, train_model.min_alpha = alpha, alpha
        with elapsed_timer() as elapsed:
            train_model.train(doc_list, total_examples=len(doc_list), epochs=1)
            duration = '%.1f' % elapsed()
            
        # Evaluate
        eval_duration = ''
        with elapsed_timer() as eval_elapsed:
            err, err_count, test_count, predictor = error_rate_for_model(train_model, train_docs, test_docs)
        eval_duration = '%.1f' % eval_elapsed()
        best_indicator = ' '
        if err <= best_error[name]:
            best_error[name] = err
            best_indicator = '*' 
        print("%s%f : %i passes : %s %ss %ss" % (best_indicator, err, epoch + 1, name, duration, eval_duration))

        if ((epoch + 1) % 5) == 0 or epoch == 0:
            eval_duration = ''
            with elapsed_timer() as eval_elapsed:
                infer_err, err_count, test_count, predictor = error_rate_for_model(train_model, train_docs, test_docs, infer=True)
            eval_duration = '%.1f' % eval_elapsed()
            best_indicator = ' '
            if infer_err < best_error[name + '_inferred']:
                best_error[name + '_inferred'] = infer_err
                best_indicator = '*'
            print("%s%f : %i passes : %s %ss %ss" % (best_indicator, infer_err, epoch + 1, name + '_inferred', duration, eval_duration))

    print('Completed pass %i at alpha %f' % (epoch + 1, alpha))
    alpha -= alpha_delta
    
print("END %s" % str(datetime.datetime.now()))

START 2017-11-30 23:33:52.614466
*0.283091 : 1 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8) 3.2s 0.2s
*0.314545 : 1 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8)_inferred 3.2s 0.8s
*0.281091 : 1 passes : Doc2Vec(dbow,d100,n5,mc2,s0.001,t8) 1.6s 0.2s
*0.370909 : 1 passes : Doc2Vec(dbow,d100,n5,mc2,s0.001,t8)_inferred 1.6s 0.4s
*0.280909 : 1 passes : Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8) 2.7s 0.2s
*0.292727 : 1 passes : Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8)_inferred 2.7s 0.5s
*0.273818 : 1 passes : dbow+dmm 0.0s 0.4s
*0.341818 : 1 passes : dbow+dmm_inferred 0.0s 0.9s
*0.275636 : 1 passes : dbow+dmc 0.0s 0.4s
*0.329091 : 1 passes : dbow+dmc_inferred 0.0s 1.2s
Completed pass 1 at alpha 0.025000
*0.282000 : 2 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8) 3.1s 0.2s
*0.278182 : 2 passes : Doc2Vec(dbow,d100,n5,mc2,s0.001,t8) 1.5s 0.2s
*0.280909 : 2 passes : Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8) 2.8s 0.2s
*0.272182 : 2 passes : dbow+dmm 0.0s 0.4s
*0.270182 : 2 passes : dbow+dmc 0.0

In [24]:
# Print best error rates achieved
print("Err rate Model")
for rate, name in sorted((rate, name) for name, rate in best_error.items()):
    print("%f %s" % (rate, name))


Err rate Model
0.261818 Doc2Vec(dbow,d100,n5,mc2,s0.001,t8)_inferred
0.265455 Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8)_inferred
0.266364 dbow+dmc
0.267091 dbow+dmm
0.276000 Doc2Vec(dbow,d100,n5,mc2,s0.001,t8)
0.278000 Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8)
0.278545 Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8)
0.281818 Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8)_inferred
0.303636 dbow+dmc_inferred
0.312727 dbow+dmm_inferred


In [25]:
doc_id = np.random.randint(simple_models[0].docvecs.count)  # Pick random doc; re-run cell for more examples
print('for doc %d...' % doc_id)
for model in simple_models:
    inferred_docvec = model.infer_vector(alldocs[doc_id].words)
    print('%s:\n %s' % (model, model.docvecs.most_similar([inferred_docvec], topn=3)))

for doc 9541...
Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8):
 [(990, 0.7450541257858276), (6155, 0.7244516015052795), (2140, 0.717074453830719)]
Doc2Vec(dbow,d100,n5,mc2,s0.001,t8):
 [(4041, 0.8820165991783142), (8378, 0.6651365756988525), (8939, 0.6619388461112976)]
Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8):
 [(4041, 0.7913916707038879), (4551, 0.764366865158081), (8779, 0.7562659978866577)]


In [26]:

import random

doc_id = np.random.randint(simple_models[0].docvecs.count)  # pick random doc, re-run cell for more examples
model = random.choice(simple_models)  # and a random model
sims = model.docvecs.most_similar(doc_id, topn=model.docvecs.count)  # get *all* similar documents
print(u'TARGET (%d): «%s»\n' % (doc_id, ' '.join(alldocs[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(alldocs[sims[index][0]].words)))



TARGET (6744): «we study the free path length and the geometric free path length in the model of the periodic two-dimensional lorentz gas ( sinai billiard ) . we give a complete and rigorous proof for the existence of their distributions in the small-scatterer limit and explicitly compute them . as a corollary one gets a complete proof for the existence of the constant term $c=2-3\ln 2+\frac{27\zeta ( 3 ) }{2\pi^2}$ in the asymptotic formula $h ( t ) =-2\ln \eps+c+o ( 1 ) $ of the ks entropy of the billiard map in this model , as conjectured by p . dahlqvist .»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dbow,d100,n5,mc2,s0.001,t8):

MOST (9502, 0.7147164344787598): «for certain real quadratic number fields , we prove density results concerning 4-ranks of tame kernels . we also discuss a relationship between 4-ranks of tame kernels and 4-class ranks of narrow ideal class groups . additionally , we give a product formula for a local hilbert symbol .»

MEDIAN (2286, 0.396057665348053): «i

In [27]:
import random

doc_id = np.random.randint(simple_models[0].docvecs.count)  # pick random doc, re-run cell for more examples
model = random.choice(simple_models)  # and a random model
sims = model.docvecs.most_similar(doc_id, topn=model.docvecs.count)  # get *all* similar documents
print(u'TARGET (%d): «%s»\n' % (doc_id, ' '.join(alldocs[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(alldocs[sims[index][0]].words)))



TARGET (4027): «we generalize the farey-brocot partition to a twodimensional continued fraction algorithm and generalized farey-brocot nets . we give an asymptotic formula for the moments of order \beta .»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8):

MOST (5883, 0.8089919686317444): «let $p ( z ) $ be a monic polynomial of degree $n$ , with complex coefficients , and let $q ( z ) $ be its monic factor . we prove an asymptotically sharp inequality of the form $\|q\|_{e} \le c^n \|p\|_e$ , where $\|\cdot\|_e$ denotes the sup norm on a compact set $e$ in the plane . the best constant $c_e$ in this inequality is found by potential theoretic methods . we also consider applications of the general result to the cases of a disk and a segment .»

MEDIAN (7354, 0.0511329285800457): «we consider the primes which divide the denominator of the x-coordinate of a sequence of rational points on an elliptic curve . it is expected that for every sufficiently large value of

In [28]:
import random

doc_id = np.random.randint(simple_models[0].docvecs.count)  # pick random doc, re-run cell for more examples
model = random.choice(simple_models)  # and a random model
sims = model.docvecs.most_similar(doc_id, topn=model.docvecs.count)  # get *all* similar documents
print(u'TARGET (%d): «%s»\n' % (doc_id, ' '.join(alldocs[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(alldocs[sims[index][0]].words)))



TARGET (1168): «we begin with a review of the structure of simple , simply-connected complex lie groups and their lie algebras , describe the chevalley lattice and the associated split group over the integers . this gives us a hyperspecial maximal compact subgroup of the p-adic lie group and we describe the other maximal parahoric subgroups and their lie algebras starting from the hyperspecial one . we then consider the killing form on the chevalley lattice and show that it is divisible by 2 times the dual coxeter number . the same holds for the lie algebras of the other maximal parahorics . we compute the discriminants of the resulting scaled forms . finally we consider jordan subgroups of the exceptional groups . we show that these jordan subgroups are globally maximal and determine their maximal compact overgroups in the p-adic lie group . the last section treats the jordan subgroups of the classical groups .»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8)